# FastAPI Application Testing

## 1. Local Testing

Uvicorn running on http://0.0.0.0:8000

In [1]:
import requests
import json
import httpx

In [ ]:
response = requests.post(
    "http://localhost:8000/chat",
    json={"message": "Show me the tables available in ENAHO database"}
)
data = response.json()
print(f"Response: {data['response']}")
print(f"Thread ID: {data['thread_id']}")

In [3]:
response = requests.post(
    "http://localhost:8000/chat",
    json={
        "message": "What columns are in the first table?",
        "thread_id": data['thread_id']
    }
)

In [ ]:
data = response.json()
print(f"Response: {data['response']}")
print(f"Thread ID: {data['thread_id']}")

In [ ]:
url = "http://localhost:8000/chat/stream"
payload = {
    "message": "De acuerdo a la GEIH, ¿qué porcentaje de hogares pagan por alquiler de su vivienda? Desagregado por departamento",
    "thread_id": "user-session-790"
}

async with httpx.AsyncClient(timeout=None) as client:
    async with client.stream("POST", url, json=payload) as response:       
        full_text = ""
        async for line in response.aiter_lines():
            if line.startswith("data: "):
                content = line[len("data: "):].strip()
                content = eval(content)["content"]
                full_text += content
                print(content, end="", flush=True)
        
        print("\n\nFull response:", full_text)

## 2. Cloud Run Testing

Docker Container running uvicorn at <BASE_URL>

In [1]:
BASE_URL = "https://esma-agent-514700908055.us-east1.run.app"

In [4]:
# Test basic connectivity
print("Testing root endpoint...")
response = requests.get(f"{BASE_URL}/")
print(json.dumps(response.json(), indent=2))

Testing root endpoint...
{
  "name": "ESMA SQL Agent API",
  "endpoints": {
    "/chat": "Non-streaming chat (POST)",
    "/chat/stream": "Streaming chat with SSE (POST)",
    "/thread/new": "Create new conversation thread (POST)",
    "/docs": "Interactive API documentation"
  }
}


In [5]:
# Test a simple query
print("\n\nTesting chat endpoint...")
response = requests.post(
    f"{BASE_URL}/chat",
    json={"message": "List the tables available in ENAHO"}
)
result = response.json()
print(f"Thread ID: {result.get('thread_id')}")
print(f"Response preview: {result.get('response', 'No response')[:500]}...")



Testing chat endpoint...
Thread ID: esma-chat-fd4dc9b3-91cb-4ee9-9823-c8e269c788be
Response preview: Of course! Here are the tables available in the ENAHO 2024 database, along with a brief description of each:

### Housing and Household Characteristics
*   **ENAHO01-2024-100 (Características de la Vivienda y del Hogar):** Contains data on housing conditions, access to basic services, and household assets.
*   **ENAHO01-2024-601 (Gastos del Hogar):** Detailed information on household expenditures, including food and non-food items.
*   **ENAHO01-2024-603 (Mantenimiento de la Vivienda):** Covers ...
